In [1]:
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.1. Summary Stats/1.1.3. Demographics Analysis/Input')
Output = ('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.1. Summary Stats/1.1.3. Demographics Analysis/Output')
cur_date = '121021'

library(readxl)
library(tidyverse)
library(reshape2)
library(car)

#reading in file
data_df <- data.frame(read_excel("CytokineData_102920.xlsx", sheet = 2))

#reading in demographics file
demographics_data_df = data.frame(read_excel("SubjectInfo_102920.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some




In [2]:
#adding demographics data to cytokine df
#could've made my life easier by just using full_join
race = c()
ethnicity = c()
sex = c()
age = c()
BMI = c()
for (i in 1:length(data_df$SubjectNo)){
    for(j in 1:length(demographics_data_df$SubjectNo)){
        if (data_df$SubjectNo[i] == demographics_data_df$SubjectNo[j]){
            race = c(race, demographics_data_df$Race[j])
            ethnicity = c(ethnicity, demographics_data_df$Ethnicity[j])
            sex = c(sex, demographics_data_df$Sex[j])
            age = c(age, demographics_data_df$Age[j])
            BMI = c(BMI, demographics_data_df$BMI[j])
        }
    }
}

data_df$Race = race
data_df$Ethnicity = ethnicity
data_df$Sex = sex
data_df$Age = age
data_df$BMI = BMI

In [3]:
head(data_df)

,Original_Identifier,Protein,Conc,Conc_pslog2,Compartment,SubjectNo,Group,SubjectID,Race,Ethnicity,Sex,Age,BMI
,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,E_C_F_002,IFNg,17.642316,4.220509,ELF,1,NS,NS_1,W,NH,F,28,37.67
2,E_C_F_002,IL10,2.873724,1.953721,ELF,1,NS,NS_1,W,NH,F,28,37.67
3,E_C_F_002,IL12p70,1.625272,1.392467,ELF,1,NS,NS_1,W,NH,F,28,37.67
4,E_C_F_002,IL13,36.117692,5.214035,ELF,1,NS,NS_1,W,NH,F,28,37.67
5,E_C_F_002,IL1b,104.409217,6.719857,ELF,1,NS,NS_1,W,NH,F,28,37.67
6,E_C_F_002,IL6,21.159536,4.469856,ELF,1,NS,NS_1,W,NH,F,28,37.67


# Chi Square Analysis/ ANOVA

Comparing demographic variables to see if there are differences across smoking statuses. These numbers are from the Table 1 demographics table.

In [4]:
head(demographics_data_df)

,SubjectNo,Group,SubjectID,Race,Ethnicity,Sex,Age,BMI,Serum_Cotinine
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,NS,NS_1,W,NH,F,28,37.67,0
2,2,NS,NS_2,O,H,F,33,35.00,0
3,3,NS,NS_3,W,NH,F,25,18.70,0
4,4,NS,NS_4,W,NH,F,26,23.00,0
5,5,NS,NS_5,As,NH,F,25,24.70,0
6,6,NS,NS_6,AA,NH,F,42,34.60,0


In [5]:
get_tables = function(df, variable){
    #getting variable stratified by smoking status
    variable_grouped = df %>% group_by(Group) %>% select(all_of(variable)) 
    #getting overall variable numbers
    overall_variable = df %>% select(all_of(variable)) 
    #adding a col that designates this from the overall cohort
    Group = rep('O', times = length(overall_variable)) #O stands for overall
    overall_variable_df = cbind(Group, overall_variable)

    variable_table = table(rbind(overall_variable_df, data.frame(variable_grouped)))
    return(variable_table)
}
sex_table = get_tables(demographics_data_df, 'Sex')
race_table = get_tables(demographics_data_df, 'Race')
ethnicity_table = get_tables(demographics_data_df, 'Ethnicity')

Adding missing grouping variables: `Group`

Adding missing grouping variables: `Group`

Adding missing grouping variables: `Group`



In [6]:
sex_table
race_table
ethnicity_table

      Sex
Group   F  M
  O    20 24
  NS    7  7
  CS    8  5
  Ecig  5 12

      Race
Group  AA As  O  W
  O    10  4  2 28
  NS    2  1  1 10
  CS    6  0  0  7
  Ecig  2  3  1 11

      Ethnicity
Group   H NH
  O     5 39
  NS    3 11
  CS    0 13
  Ecig  2 15

In [7]:
#creating a matrix for previous smoking history since this data was in the OG dataset
previous_smoking_history = matrix(c(25, 0, 13, 12, 19, 14, 4, 5), ncol = 2)
colnames(previous_smoking_history) = c("Y", "N")
rownames(previous_smoking_history) = c("O", "NS", "Ecig", "CS")
previous_smoking_history

,Y,N
O,25,19
NS,0,14
Ecig,13,4
CS,12,5


In [8]:
#fisher exact test
fisher.test(sex_table)
fisher.test(race_table)
fisher.test(ethnicity_table)
fisher.test(previous_smoking_history)


	Fisher's Exact Test for Count Data

data:  sex_table
p-value = 0.3703
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data

data:  race_table
p-value = 0.5351
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data

data:  ethnicity_table
p-value = 0.4026
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data

data:  previous_smoking_history
p-value = 1.711e-05
alternative hypothesis: two.sided


In [9]:
#dichotimized race
#making race binary between white and non-white subjects
for (i in 1:length(demographics_data_df$Race)){
    if(demographics_data_df$Race[i] != 'W'){
        demographics_data_df$Race[i] = 'NW'
    }
}

#dichotimized race
di_race_table = get_tables(demographics_data_df, 'Race')
fisher.test(di_race_table)

Adding missing grouping variables: `Group`




	Fisher's Exact Test for Count Data

data:  di_race_table
p-value = 0.8293
alternative hypothesis: two.sided


Since none of the p values are significant, the demographics between each smoking status isn't statistically different.

In [10]:
#running anova test for continuous values: age, bmi, cotinine

#putting group into a factor in order to make comparisons
demographics_data_df$Group = factor(demographics_data_df$Group, labels = c(unique(demographics_data_df$Group)))

#normalizing data
mutated_df = demographics_data_df %>%
    mutate(Norm_Age = scale(Age), Norm_BMI = scale(BMI), Norm_Cotinine = scale(Serum_Cotinine))
#contrasts show what compartments are being compared in anova
#these comparisons aren't what we want so I changed them in the function below
contrasts(mutated_df$Group)

,CS,Ecig
NS,0,0
CS,1,0
Ecig,0,1


                    Table of Contrasts
>                   NS    | CS  |  Ecig  |  Sum
>     Contrast 1 |  -2    |  1  |   1    |   0
>     Contrast 2 |  0     | -1  |   1    |   0

> - Contrast 1: compares NS to all other groups
> - Contrast 2: compares CS and Ecig

In [11]:
contrasts(mutated_df$Group) = cbind(c(-2,1,1),c(0,-1,1)) #meaning is specified above
anova_age = Anova(aov(Norm_Age~Group, data = mutated_df), type = 'III')
anova_age
anova_bmi = Anova(aov(Norm_BMI~Group, data = mutated_df), type = 'III')
anova_bmi
anova_continine = Anova(aov(Norm_Cotinine~Group, data = mutated_df), type = 'III')
anova_continine

,Sum Sq,Df,F value,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.05208576,1,0.05517635,0.8154583
Group,4.29652993,2,2.27573557,0.1155499
Residuals,38.70347007,41,NA,NA


,Sum Sq,Df,F value,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.003458646,1,0.003394073,0.9538327
Group,1.239001803,2,0.607934966,0.5494275
Residuals,40.760998197,40,NA,NA


,Sum Sq,Df,F value,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),2.396099e-06,1,3.612112e-06,9.984928e-01
Group,1.580260e+01,2,1.191119e+01,8.350401e-05
Residuals,2.719740e+01,41,NA,NA
